In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
import pandas as pd
import datetime
!pip install transformers

In [10]:
data = pd.read_json("/kaggle/input/ineuron-hackathon/messages.json")

In [11]:
data.columns

In [12]:
msgs = data['conversations'][1]['MessageList'][:100]

In [13]:
msgs[0]

In [14]:
res=[]

In [15]:
import re
def remove_files(data):
    if 'https' in data or '<' in data or '/>' in data or '?' in data or len(data)==0 or 'Call Log' in data:
          return 'null'
    return data

In [26]:
res=[]

In [27]:
for i in range(1000):
    msgs = data['conversations'][i]['MessageList']
    for msg in msgs:
        temp = []
        if msg['displayName']!= None:
            temp.append('question')
        else:
            temp.append('answer')
        val =remove_files(msg['content'])
        if val!='null':
            temp.append(val)
            temp.append(msg['originalarrivaltime'])
            temp.append(msg['conversationid'])
            res.append(temp)


In [28]:
res[0]

In [29]:
temp = pd.DataFrame(res, columns = ['type', 'message','time','id'])

In [30]:
temp.info()

In [31]:
temp['time']=temp['time'].astype('datetime64')

In [32]:
temp = temp.groupby(['time','type','id']).sum()

In [34]:
!pip install openpyxl

In [35]:
temp.to_excel("final_data.xlsx")

In [36]:
data = pd.read_excel("final_data.xlsx")

In [37]:
data.head()

In [38]:
data['type_match'] = data.type.eq(data.type.shift(-1))

In [39]:
data.head(20)

In [40]:
data.head()

In [41]:
data = data[data.type_match != True]

In [42]:
data.shape

In [43]:
dd = data[['type','message']]

In [44]:
context = [' .'.join([str(i) for i in data['message'].loc[data['type']=='question'].values])]

In [45]:
data['type'] = data['type']+":"

In [46]:
chat = data[['type','message']].values

In [47]:
dd.reset_index(inplace=True)

In [48]:
dd = dd[1:]

In [49]:
ques = dd['message'].loc[dd['type']=='question'].values

In [50]:
ans = dd['message'].loc[dd['type']=='answer'].values

In [51]:
res = []
t = {}
ff ={}
final =[]
for i in range(len(ans)-1):
    ff['context'] = str(ques[i]) +'. '+ str(ans[i])
    t['question'] = str(ques[i])
    t['answer'] = [{"text":str(ans[i]),
                  "answer_start":ff['context'].index(str(ans[i]))}]
    res.append(t)
    ff['qas']=res
  
    final.append(ff)
    t={}
    ff={}
    res=[]

In [52]:
data = {"data":
    [
        {"title": "Chat Bot",
         "paragraphs":final}]}

In [53]:
def read_squad(squad_dict):
    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answer']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    return contexts[:50000], questions[:50000], answers[:50000],contexts[50000:], questions[50000:], answers[50000:]

train_contexts, train_questions, train_answers,val_contexts, val_questions, val_answers = read_squad(data)

In [54]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [55]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [56]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        # if None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    {key: train_encodings[key] for key in ['input_ids', 'attention_mask']},
    {key: train_encodings[key] for key in ['start_positions', 'end_positions']}
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    {key: val_encodings[key] for key in ['input_ids', 'attention_mask']},
    {key: val_encodings[key] for key in ['start_positions', 'end_positions']}
))

In [ ]:
from transformers import TFDistilBertForQuestionAnswering
model = TFDistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

In [ ]:
train_dataset = train_dataset.map(lambda x, y: (x, (y['start_positions'], y['end_positions'])))
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.distilbert.return_dict = False 

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=loss) 
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=200)